**DAPT on ClimateBERT for familiarity with contracts**

This is an artifact to explain how training was done. Paths may be broken and it will take a long, long time to run even on a GPU. Therefore it is not cleaned but is included for completeness.

In [ ]:
#Note: You will need a GPU to run these cells

In [ ]:
import os
from pathlib import Path
import random
import shutil
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer


In [ ]:
import wandb
wandb.init(project="huggingface", name="climatebert-dapt-contracts")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ger23 (ger23-imperial-college-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA available: True
Device count: 1
Device name: Tesla T4


In [ ]:
!pip install "numpy<2.0"

In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
model_name = "climatebert/distilroberta-base-climate"
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")
model = AutoModelForMaskedLM.from_pretrained("climatebert/distilroberta-base-climate-f")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile
contract_path = '/content/drive/MyDrive/contracts.tar.gz'

# Extract into a working directory
extract_path = '/content/contracts'

with tarfile.open(contract_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

# Verify extraction
import os
print(os.listdir(extract_path))


KeyboardInterrupt: 

In [ ]:
source_root = Path("/content/contracts")
target_root = Path("/content/sample_contracts")
target_root.mkdir(exist_ok=True)

In [ ]:
for year_folder in sorted(source_root.iterdir()):
    if year_folder.is_dir():
        all_files = list(year_folder.glob("*.txt"))
        if not all_files:
            continue
        sample_size = max(1, int(0.1 * len(all_files)))
        sampled_files = random.sample(all_files, sample_size)

        for file_path in sampled_files:
            # Give each sampled file a unique name
            new_name = f"{year_folder.name}_{file_path.name}"
            shutil.copy(file_path, target_root / new_name)

In [ ]:
from datasets import load_dataset
import glob

sampled_files = glob.glob("/content/sample_contracts/*.txt")

dataset = load_dataset("text", data_files={"train": sampled_files})


Resolving data files:   0%|          | 0/65074 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import shutil

# Delete the existing sample folder if it exists
shutil.rmtree("/content/sample_data", ignore_errors=True)
shutil.rmtree("/content/contracts", ignore_errors=True)


In [ ]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(x["text"], truncation=True, padding=False, max_length=512),
    batched=True,
    remove_columns=["text"]
)


Map:   0%|          | 0/84179689 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk("/content/drive/MyDrive/tokenized_contracts")

NameError: name 'tokenized_dataset' is not defined

In [ ]:
from datasets import load_from_disk

tokenized_dataset = load_from_disk("/content/drive/MyDrive/tokenized_contracts")

In [ ]:
print(tokenized_dataset)
print(tokenized_dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 84179689
    })
})
{'input_ids': [0, 50141, 2], 'attention_mask': [1, 1, 1]}


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./climatebert-dapt-contracts",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=True,
    save_safetensors=False
)


In [ ]:
from transformers import TrainerCallback
class WandbCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        if os.path.exists(checkpoint_dir):
            artifact = wandb.Artifact(
                name=f"climatebert-checkpoint-{state.global_step}",
                type="model"
            )
            artifact.add_dir(checkpoint_dir)
            wandb.log_artifact(artifact)
            print(f"\n📦 Logged checkpoint-{state.global_step} to Weights & Biases.")


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[WandbCheckpointCallback()],
)

<ipython-input-15-280656486936>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,2.410400
1000,2.165600
1500,2.077800
2000,2.020000
2500,1.993900
3000,1.946500
3500,1.907000
4000,1.896100
4500,1.856400
5000,1.852200


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-1000)... Done. 9.1s



📦 Logged checkpoint-1000 to Weights & Biases.


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-2000)... Done. 3.0s


...

wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-234000)... Done. 4.8s



📦 Logged checkpoint-234000 to Weights & Biases.


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-235000)... Done. 4.8s



📦 Logged checkpoint-235000 to Weights & Biases.


Step,Training Loss
500,2.410400
1000,2.165600
1500,2.077800
2000,2.020000
2500,1.993900
3000,1.946500
3500,1.907000
4000,1.896100
4500,1.856400
5000,1.852200




---



In [ ]:
artifact = wandb.use_artifact("ger23-imperial-college-london/huggingface/climatebert-checkpoint-235000:v0", type="model")
checkpoint_dir = artifact.download()


wandb: Downloading large artifact climatebert-checkpoint-235000:v0, 947.33MB. 14 files... 
wandb:   14 of 14 files downloaded.  
Done. 0:0:4.7


In [ ]:
trainer.train(resume_from_checkpoint=checkpoint_dir)

You are resuming training from a checkpoint trained with 4.51.1 of Transformers but your current version is 4.51.3. This is not recommended and could yield to errors or unwanted behaviors.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
235500,1.271400
236000,1.272000
236500,1.272900
237000,1.280900
237500,1.299800
238000,1.278000
238500,1.271200
239000,1.286900
239500,1.275700
240000,1.269500


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-236000)... Done. 2.5s



📦 Logged checkpoint-236000 to Weights & Biases.


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-237000)... Done. 2.7s


...


📦 Logged checkpoint-304000 to Weights & Biases.


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-305000)... Done. 3.9s



📦 Logged checkpoint-305000 to Weights & Biases.


wandb: Adding directory to artifact (./climatebert-dapt-contracts/checkpoint-306000)... Done. 2.4s



📦 Logged checkpoint-306000 to Weights & Biases.




---



# Classification Head

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Clause-Comparison/')

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tclp.clause_detector import detector_utils as du
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score, accuracy_score
import numpy as np


In [ ]:
os.chdir('/content/drive/MyDrive/Clause-Comparison/tclp/clause_detector/notebooks')

In [ ]:
os.environ["WANDB_API_KEY"] = "3bff39257044a210c516c50b1d45e18d506dc4ee"

In [ ]:
import wandb



---



In [ ]:
modified_data_folder = '../../data/synth_data/modified_real'
untouched_data_folder = '../../data/synth_data/untouched'
modified_gen_data_folder = '../../data/synth_data/modified_gen'

In [ ]:
texts, labels, contract_ids, contract_level_labels = du.load_labeled_contracts(
    modified_data_folder, modified=True
)
(
    texts_untouched,
    labels_untouched,
    contract_ids_untouched,
    contract_level_labels_untouched,
) = du.load_labeled_contracts(untouched_data_folder)
texts_gen, labels_gen, contract_ids_gen, contract_level_labels_gen = (
    du.load_labeled_contracts(modified_gen_data_folder, modified=True)
)

In [ ]:
# combine texts from different sources
texts = texts + texts_untouched + texts_gen
labels = labels + labels_untouched + labels_gen
contract_ids = contract_ids + contract_ids_untouched + contract_ids_gen
contract_level_labels = (
    contract_level_labels + contract_level_labels_untouched + contract_level_labels_gen
)

In [ ]:
data = du.create_and_clean_base_df(texts, labels, contract_ids, contract_level_labels)

In [ ]:
clause_folder = "../../data/clause_boxes"

In [ ]:
clause_texts, clause_labels, clause_ids, clause_reality = du.load_clauses(clause_folder)

In [ ]:
# put this in the same form as the contract data so they can be combined; contract label for all of them is 1
clause_data = pd.DataFrame(
    {
        "contract_ids": clause_ids,
        "text": clause_texts,
        "label": clause_labels,
        "contract_label": [1] * len(clause_ids),
        "real_clause": clause_reality,
    }
)

In [ ]:
full_data = pd.concat([data, clause_data], ignore_index=True)



---



In [ ]:
train_data, val_data, test_data, train_indices, val_indices, test_indices = (
    du.custom_train_test_split(full_data, "real_clause")
)

NameError: name 'full_data' is not defined

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data[["text", "label"]].reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_data[["text", "label"]].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_data[["text", "label"]].reset_index(drop=True))

In [ ]:
save_path = '../../train_test_val'

In [ ]:
train_data.to_csv(save_path + "train_data.csv", index=False)
val_data.to_csv(save_path + "val_data.csv", index=False)
test_data.to_csv(save_path + "test_data.csv", index=False)



---



In [ ]:
model_path = "../../contract_climate_bert"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=2  # binary classification: clause / not-clause
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../../contract_climate_bert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset

In [ ]:
#load in the train_test_val
save_path = '../../train_test_val'

train_dataset = pd.read_csv(save_path + "train_data.csv")
val_dataset = pd.read_csv(save_path + "val_data.csv")
test_dataset = pd.read_csv(save_path + "test_data.csv")

train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [ ]:
def tokenize(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True).rename_column("label", "labels")
val_dataset = val_dataset.map(tokenize, batched=True).rename_column("label", "labels")
test_dataset = test_dataset.map(tokenize, batched=True).rename_column("label", "labels")

for ds in [train_dataset, val_dataset, test_dataset]:
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/750135 [00:00<?, ? examples/s]

Map:   0%|          | 0/99691 [00:00<?, ? examples/s]

Map:   0%|          | 0/148748 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
help(TrainingArguments.__init__)


Help on function __init__ in module transformers.training_args:

__init__(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Unio

In [ ]:
wandb.init(
    project="climatebert-classification",
    name="contract-classification-run-10",
    config={
        "learning_rate": 2e-5,
        "batch_size": 8,
        "epochs": 3,
        "logging_steps": 1000,
        "eval_steps": 10000,
        "save_steps": 10000
    }
)

# Step 2: Watch the model (optional, logs weights/gradients)
wandb.watch(model, log="all", log_freq=100)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    eval_steps=10000,
    save_strategy="steps",
    save_steps=10000,
    logging_strategy="steps",
    logging_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    prediction_loss_only=False,
    save_total_limit=3,
    report_to="wandb",
    run_name="contract-classification-run-3"
)

# Step 4: Define metrics
def compute_metrics(eval_pred):
    import numpy as np
    from sklearn.metrics import accuracy_score, f1_score
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
    }

# Step 5: Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[WandbCheckpointCallback()]
)

<ipython-input-49-e3b83dd239c8>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
10000,0.028700,0.047245,0.992186,0.944440


wandb: Adding directory to artifact (./results/checkpoint-10000)... Done. 2.3s



📦 Logged checkpoint-10000 to Weights & Biases.


Step,Training Loss,Validation Loss,Accuracy,F1
10000,0.028700,0.047245,0.992186,0.944440


In [ ]:
artifact = wandb.use_artifact("ger23-imperial-college-london/climatebert-classification/climatebert-checkpoint-60000:v0", type="model")
checkpoint_dir = artifact.download()


wandb: Downloading large artifact climatebert-checkpoint-60000:v0, 946.66MB. 13 files... 
wandb:   13 of 13 files downloaded.  
Done. 0:0:1.6 (590.3MB/s)


In [ ]:
scaler_path = os.path.join(checkpoint_dir, "scaler.pt")
if os.path.exists(scaler_path):
    os.remove(scaler_path)


In [ ]:
trainer.train(resume_from_checkpoint=checkpoint_dir)

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.